In [1]:
import pandas as pd
import jieba

In [2]:
sms_data = pd.read_csv('data/80w.txt', sep='\t', header=None)
sms_data.head()

,0,1,2
0,1,0,商业秘密的秘密性那是维系其商业价值和垄断地位的前提条件之一
1,2,1,南口阿玛施新春第一批限量春装到店啦   春暖花开淑女裙、冰蓝色公主衫 ...
2,3,0,带给我们大常州一场壮观的视觉盛宴
3,4,0,有原因不明的泌尿系统结石等
4,5,0,23年从盐城拉回来的麻麻的嫁妆


第一列为原始序号，第二列为短信的分类，0表示正常短信，1表示垃圾短信，第三列就是短信的正文。

In [3]:
print(sms_data.shape)

(754843, 3)


In [4]:
# stop words
stop_words = pd.read_csv('data/stopWord.txt', header=None)
stop_words.head()

,0
0,、
1,，
2,。
3,？
4,喔


In [5]:
STOP_WORDS = frozenset((stop_words[0].values))
def participle(text):
    seg_list = jieba.cut(text)
    results = set(seg_list) - STOP_WORDS
    return ' '.join(results)

In [6]:
sms_data['participle'] = sms_data[2].apply(participle)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.963 seconds.
Prefix dict has been built succesfully.


In [7]:
sms_data.head()

,0,1,2,participle
0,1,0,商业秘密的秘密性那是维系其商业价值和垄断地位的前提条件之一,商业秘密 地位 秘密性 是 那 商业价值 之一 其 前提条件 垄断 的 和 维系
1,2,1,南口阿玛施新春第一批限量春装到店啦   春暖花开淑女裙、冰蓝色公主衫 ...,衫 皇 气质 公主 长半裙 女王 裙 阿玛施  南口 到 店  限量  新春 西装...
2,3,0,带给我们大常州一场壮观的视觉盛宴,一场 视觉 带给 我们 常州 大 壮观 的 盛宴
3,4,0,有原因不明的泌尿系统结石等,不明 泌尿系统 有 原因 结石 等 的
4,5,0,23年从盐城拉回来的麻麻的嫁妆,年 23 拉回来 盐城 从 的 麻麻 嫁妆


In [8]:
X = sms_data['participle'].values
Y = sms_data[1].values

In [9]:
print(Y)

[0 1 0 ... 0 0 0]


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import shuffle

random_seed = 17
X_cross_train, X_cross_test, Y_cross_train, Y_cross_test = train_test_split(X, Y, test_size=0.3, random_state=random_seed)
#X_cross_train, y_cross_train = shuffle(X_cross_train, Y_cross_train, random_state=random_seed)

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

vectorizer = CountVectorizer()
X_train_termcounts = vectorizer.fit_transform(X_cross_train)

tfidf_tansformer = TfidfTransformer()
X_train_tfidf = tfidf_tansformer.fit_transform(X_train_termcounts)

In [12]:
## Transform persistence
import pickle

# Save to file in the current working directory
pkl_filename = "vectorizer.pkl"
with open(pkl_filename, 'wb') as file:  
    pickle.dump(vectorizer, file)
    
pkl_filename = "transformer.pkl"
with open(pkl_filename, 'wb') as file:  
    pickle.dump(tfidf_tansformer, file)

In [13]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB

classifier = MultinomialNB().fit(X_train_tfidf, Y_cross_train)

In [14]:
## Model persistence

# Save to file in the current working directory
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:  
    pickle.dump(classifier, file)

In [15]:
X_test_termcounts = vectorizer.transform(X_cross_test)
X_test_tfidf = tfidf_tansformer.transform(X_test_termcounts)

predicted_categories = classifier.predict(X_test_tfidf)

In [16]:
print(X_cross_test)

['整理 细胞 的 了 好 自己 照片 x 今天' '手胃 我 细胞 每 一个 浑身 汗毛 我心 脑子 都 不 在 的 开心'
 '这 就 近墨者黑 哦 近朱者赤 叫做' ... '为 股份 证券 营业部 有限公司 海通 ： 卖方 文化路 杭州'
 '飞机 了 睡觉 上 睡 完全 好 得 在 不想 没' '美女 小偷 一个 了 三个 大盗 影片 神秘 和 讲述']


In [17]:
from sklearn.metrics import accuracy_score

accuracy_score(Y_cross_test, predicted_categories)

0.983744971362711

In [18]:
category_map = {
    0: 'normal',
    1: 'spam'
}

for sentence, category, real in zip(X_cross_test[:100], predicted_categories[:100], Y_cross_test[:100]):
    print('\nsms_content: ', sentence, '\nPredicted: ', category_map[category], '\ntrue value: ', category_map[real])


sms_content:  整理 细胞 的 了 好 自己 照片 x 今天 
Predicted:  normal 
true value:  normal

sms_content:  手胃 我 细胞 每 一个 浑身 汗毛 我心 脑子 都 不 在 的 开心 
Predicted:  normal 
true value:  normal

sms_content:  这 就 近墨者黑 哦 近朱者赤 叫做 
Predicted:  normal 
true value:  normal

sms_content:  我 公车私用 举报 是 怕 警车 大概 小区 夜停 
Predicted:  normal 
true value:  normal

sms_content:  笔记本电脑 桌子 发生爆炸 烧焦 将 突然 自己 的 联想 
Predicted:  normal 
true value:  normal

sms_content:  项目 纹唇 水光针 纹眉 管理 永久 咨询 MTS 眼线 等 店内 美白针 纹 半 皮肤 可 
Predicted:  normal 
true value:  normal

sms_content:  反弹 等 权重股 带动 保险 有所 下 的 
Predicted:  normal 
true value:  normal

sms_content:  我 飞机 了 半路上 提示 延误 手机 
Predicted:  normal 
true value:  normal

sms_content:  永久 XP 保护 承诺 电脑 侵害 免受 
Predicted:  normal 
true value:  normal

sms_content:  多余 不该 花千骨 有 欢乐 小说 觉得 后面 一直 的 
Predicted:  normal 
true value:  normal

sms_content:  美女 韩国 几乎 人手 一支 
Predicted:  normal 
true value:  normal

sms_content:  背心 吊带 半截 丝 安详 生活 民族村 + 慢 
Predicted:  normal 
true value:  normal

sms_content:  